# Peformance Comparison of cuML and scikit-learn for regression

This notebook compares the performance of cuML and scikit-learn. The comparisons are conducted on datasets of identical sizes. The notebook primarily demonstrates the speedup factor that users can experience when similar machine learning algorithms from scikit-learn are run on GPUs using cuML.



# Introduction to cuML

cuML is a powerful suite of machine learning algorithms designed to accelerate data science and analytical tasks using GPUs. It provides practitioners with a familiar API similar to scikit-learn's, making it easy to fit, predict, and transform data without needing to program directly on a GPU.

## Why cuML?

As datasets continue to grow larger, traditional CPU-based algorithms can become slow and inefficient. cuML offers a solution by leveraging the parallel processing power of GPUs. With cuML, data is initially loaded onto the GPU, enabling compute tasks to be performed directly on the GPU, resulting in faster and more efficient processing.

## Key Features

- **GPU Acceleration**: cuML harnesses the computational power of GPUs to accelerate machine learning tasks.
- **Familiar API**: The API of cuML closely mirrors that of scikit-learn, making it easy for users to transition to GPU-accelerated workflows.
- **Streamlined Workflow**: With cuML, users can seamlessly perform common tasks such as fitting models, making predictions, and transforming data, all without the need to write GPU-specific code.

## Use Cases

- **Large Datasets**: cuML is particularly well-suited for working with large datasets where traditional CPU-based algorithms may struggle to scale.
- **High-Performance Computing**: Users can leverage cuML for high-performance computing tasks requiring fast and efficient machine learning algorithms.
- **Data Science and Analytics**: Whether it's exploratory data analysis, model training, or predictive analytics, cuML empowers data scientists and analysts to work more efficiently and effectively.

In summary, cuML offers a fast and efficient solution for machine learning on GPUs, providing practitioners with the tools they need to tackle large-scale data science and analytical tasks.


In [1]:
import cuml
from cupy import asnumpy
from joblib import dump, load

In [2]:
from cuml.datasets import make_regression
from cuml.model_selection import train_test_split
from cuml.linear_model import LinearRegression as cuLR
from sklearn.metrics import r2_score

In [3]:
import time 

n_samples = 100000
n_features = 256
n_info = 70

X_reg, y_reg = make_regression( n_samples = n_samples,
                                n_features = n_features,
                                n_informative = n_info,
                                random_state = 123 )
print(f'Shape of X_reg {X_reg.shape} and y_reg {y_reg.shape} and Type {X_reg.dtype, y_reg.dtype} \n')

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split( X_reg,
                                                                     y_reg,
                                                                     train_size = 0.8,
                                                                     random_state = 10 )
cuml_reg_model = cuLR( fit_intercept = True,
                       normalize = True,
                       algorithm = 'eig' ) # use an eigendecomposition of the covariance matrix;

start_time = time.time()
trained_LR = cuml_reg_model.fit( X_reg_train, y_reg_train )
end_time = time.time()

# Time taken for cuML model training
cuml_training_time = end_time - start_time

cu_preds = trained_LR.predict( X_reg_test )

cu_r2 = cuml.metrics.r2_score( y_reg_test, cu_preds )
sk_r2 = r2_score( asnumpy( y_reg_test ), asnumpy( cu_preds ) )

print("cuml's r2 score : ", cu_r2)
print("sklearn's r2 score : ", sk_r2)
print("cuML's training time:", cuml_training_time, "seconds")


# save and reload
dump( trained_LR, 'LR.model')

# to reload the model uncomment the line below
# loaded_model = load('LR.model')

Shape of X_reg (100000, 256) and y_reg (100000, 1) and Type (dtype('float32'), dtype('float32')) 

cuml's r2 score :  1.0
sklearn's r2 score :  0.999999999994969
cuML's training time: 0.018104076385498047 seconds


/home/yerzhan/.local/lib/python3.10/site-packages/cuml/internals/api_decorators.py:382: UserWarning: Starting from version 23.08, the new 'copy_X' parameter defaults to 'True', ensuring a copy of X is created after passing it to fit(), preventing any changes to the input, but with increased memory usage. This represents a change in behavior from previous versions. With `copy_X=False` a copy might still be created if necessary. Explicitly set 'copy_X' to either True or False to suppress this warning.
  return init_func(self, *args, **filtered_kwargs)


['LR.model']

## Scikit-learn Linear Regression model on CPU

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np
import time

n_samples = 100000
n_features = 256
n_info = 70

X_reg, y_reg = make_regression(n_samples=n_samples,
                                n_features=n_features,
                                n_informative=n_info,
                                random_state=123)

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(asnumpy(X_reg),
                                                                    asnumpy(y_reg),
                                                                    train_size=0.8,
                                                                    random_state=10)

start_time = time.time()  

sk_reg_model = LinearRegression()
sk_reg_model.fit(X_reg_train, y_reg_train)

end_time = time.time()  


sk_learning_time = end_time - start_time


sk_preds = sk_reg_model.predict(X_reg_test)


sk_r2 = r2_score(y_reg_test, sk_preds)


print("scikit-learn's r2 score:", sk_r2)
print("scikit-learn's training time:", sk_learning_time, "seconds")


scikit-learn's r2 score: 0.9999999999977812
scikit-learn's training time: 0.5428879261016846 seconds


## Results comparison

cuML's training time: **~0.018 seconds**

cpu's training time of scikit learn linear regression model: **~0.543 seconds**

cuML training is approximately `**30 times faster**` than scikit-learn regression model on 100K samples and 256 features, which corresponds to the acceleration table on the [rapids.ai website](https://rapids.ai/)